Trains a ResNet on the CIFAR10 dataset.

ResNet v1:
[Deep Residual Learning for Image Recognition
](https://arxiv.org/pdf/1512.03385.pdf)

ResNet v2:
[Identity Mappings in Deep Residual Networks
](https://arxiv.org/pdf/1603.05027.pdf)


Model|n|200-epoch accuracy|Original paper accuracy |sec/epoch GTX1080Ti
:------------|--:|-------:|-----------------------:|---:
ResNet20   v1|  3| 92.16 %|                 91.25 %|35
ResNet32   v1|  5| 92.46 %|                 92.49 %|50
ResNet44   v1|  7| 92.50 %|                 92.83 %|70
ResNet56   v1|  9| 92.71 %|                 93.03 %|90
ResNet110  v1| 18| 92.65 %|            93.39+-.16 %|165
ResNet164  v1| 27|     - %|                 94.07 %|  -
ResNet1001 v1|N/A|     - %|                 92.39 %|  -

&nbsp;

Model|n|200-epoch accuracy|Original paper accuracy |sec/epoch GTX1080Ti
:------------|--:|-------:|-----------------------:|---:
ResNet20   v2|  2|     - %|                     - %|---
ResNet32   v2|N/A| NA    %|            NA         %| NA
ResNet44   v2|N/A| NA    %|            NA         %| NA
ResNet56   v2|  6| 93.01 %|            NA         %|100
ResNet110  v2| 12| 93.15 %|            93.63      %|180
ResNet164  v2| 18|     - %|            94.54      %|  -
ResNet1001 v2|111|     - %|            95.08+-.14 %|  -

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)

/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sunxin/minico

1.13.2
2.2.4


Using TensorFlow backend.


In [2]:
import numpy as np
from keras.datasets import cifar10
from keras import models
from keras import layers
from keras import regularizers
from keras import optimizers
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

In [3]:
# Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 200
data_augmentation = True
num_classes = 10

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 2

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 2

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [4]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [5]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = layers.Conv2D(num_filters,
                         kernel_size=kernel_size,
                         strides=strides,
                         padding='same',
                         kernel_initializer='he_normal',
                         kernel_regularizer=regularizers.l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = layers.BatchNormalization()(x)
        if activation is not None:
            x = layers.Activation(activation)(x)
    else:
        if batch_normalization:
            x = layers.BatchNormalization()(x)
        if activation is not None:
            x = layers.Activation(activation)(x)
        x = conv(x)
    return x

In [6]:
def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = layers.Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.AveragePooling2D(pool_size=8)(x)
    y = layers.Flatten()(x)
    outputs = layers.Dense(num_classes,
                           activation='softmax',
                           kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

In [7]:
if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

Instructions for updating:
Colocations handled automatically by placer.
Learning rate:  0.001
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
_______________

In [8]:
# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

In [9]:
# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=int(np.ceil(x_train.shape[0] / batch_size)),
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, workers=4,
                        callbacks=callbacks)

Using real-time data augmentation.
Instructions for updating:
Use tf.cast instead.
Epoch 1/200
Learning rate:  0.001
1563/1563 [==============================] - 50s 32ms/step - loss: 1.7496 - acc: 0.4903 - val_loss: 1.4911 - val_acc: 0.5642

Epoch 00001: val_acc improved from -inf to 0.56420, saving model to /home/sunxin/qhm_experiment/experiments/saved_models/cifar10_ResNet20v2_model.001.h5
Epoch 2/200
Learning rate:  0.001
1563/1563 [==============================] - 44s 28ms/step - loss: 1.3360 - acc: 0.6209 - val_loss: 1.3245 - val_acc: 0.6200

Epoch 00002: val_acc improved from 0.56420 to 0.62000, saving model to /home/sunxin/qhm_experiment/experiments/saved_models/cifar10_ResNet20v2_model.002.h5
Epoch 3/200
Learning rate:  0.001
1563/1563 [==============================] - 44s 28ms/step - loss: 1.1622 - acc: 0.6799 - val_loss: 1.1625 - val_acc: 0.6876

Epoch 00003: val_acc improved from 0.62000 to 0.68760, saving model to /home/sunxin/qhm_experiment/experiments/saved_models/cifa

1563/1563 [==============================] - 44s 28ms/step - loss: 0.6129 - acc: 0.8602 - val_loss: 0.6862 - val_acc: 0.8391

Epoch 00031: val_acc did not improve from 0.84750
Epoch 32/200
Learning rate:  0.001
1563/1563 [==============================] - 44s 28ms/step - loss: 0.6123 - acc: 0.8609 - val_loss: 0.8594 - val_acc: 0.8073

Epoch 00032: val_acc did not improve from 0.84750
Epoch 33/200
Learning rate:  0.001
1563/1563 [==============================] - 44s 28ms/step - loss: 0.6041 - acc: 0.8659 - val_loss: 0.6873 - val_acc: 0.8446

Epoch 00033: val_acc did not improve from 0.84750
Epoch 34/200
Learning rate:  0.001
1563/1563 [==============================] - 45s 29ms/step - loss: 0.6021 - acc: 0.8647 - val_loss: 0.7679 - val_acc: 0.8200

Epoch 00034: val_acc did not improve from 0.84750
Epoch 35/200
Learning rate:  0.001
1563/1563 [==============================] - 44s 28ms/step - loss: 0.5992 - acc: 0.8670 - val_loss: 0.7029 - val_acc: 0.8338

Epoch 00035: val_acc did not i

1563/1563 [==============================] - 44s 28ms/step - loss: 0.5265 - acc: 0.8897 - val_loss: 0.7028 - val_acc: 0.8412

Epoch 00068: val_acc did not improve from 0.86090
Epoch 69/200
Learning rate:  0.001
1563/1563 [==============================] - 45s 28ms/step - loss: 0.5255 - acc: 0.8885 - val_loss: 0.8178 - val_acc: 0.8231

Epoch 00069: val_acc did not improve from 0.86090
Epoch 70/200
Learning rate:  0.001
1563/1563 [==============================] - 44s 28ms/step - loss: 0.5267 - acc: 0.8903 - val_loss: 0.7516 - val_acc: 0.8205

Epoch 00070: val_acc did not improve from 0.86090
Epoch 71/200
Learning rate:  0.001
1563/1563 [==============================] - 45s 29ms/step - loss: 0.5177 - acc: 0.8917 - val_loss: 0.7699 - val_acc: 0.8268

Epoch 00071: val_acc did not improve from 0.86090
Epoch 72/200
Learning rate:  0.001
1563/1563 [==============================] - 44s 28ms/step - loss: 0.5192 - acc: 0.8914 - val_loss: 0.7141 - val_acc: 0.8367

Epoch 00072: val_acc did not i

1563/1563 [==============================] - 44s 28ms/step - loss: 0.2736 - acc: 0.9584 - val_loss: 0.4421 - val_acc: 0.9116

Epoch 00102: val_acc did not improve from 0.91230
Epoch 103/200
Learning rate:  0.0001
1563/1563 [==============================] - 45s 29ms/step - loss: 0.2715 - acc: 0.9583 - val_loss: 0.4394 - val_acc: 0.9082

Epoch 00103: val_acc did not improve from 0.91230
Epoch 104/200
Learning rate:  0.0001
1563/1563 [==============================] - 44s 28ms/step - loss: 0.2677 - acc: 0.9596 - val_loss: 0.4439 - val_acc: 0.9093

Epoch 00104: val_acc did not improve from 0.91230
Epoch 105/200
Learning rate:  0.0001
1563/1563 [==============================] - 44s 28ms/step - loss: 0.2634 - acc: 0.9613 - val_loss: 0.4403 - val_acc: 0.9108

Epoch 00105: val_acc did not improve from 0.91230
Epoch 106/200
Learning rate:  0.0001
1563/1563 [==============================] - 44s 28ms/step - loss: 0.2621 - acc: 0.9616 - val_loss: 0.4400 - val_acc: 0.9102

Epoch 00106: val_acc d


Epoch 00137: val_acc did not improve from 0.91620
Epoch 138/200
Learning rate:  1e-05
1563/1563 [==============================] - 45s 29ms/step - loss: 0.2013 - acc: 0.9762 - val_loss: 0.4178 - val_acc: 0.9148

Epoch 00138: val_acc did not improve from 0.91620
Epoch 139/200
Learning rate:  1e-05
1563/1563 [==============================] - 44s 28ms/step - loss: 0.2028 - acc: 0.9749 - val_loss: 0.4185 - val_acc: 0.9153

Epoch 00139: val_acc did not improve from 0.91620
Epoch 140/200
Learning rate:  1e-05
1563/1563 [==============================] - 45s 29ms/step - loss: 0.2023 - acc: 0.9753 - val_loss: 0.4186 - val_acc: 0.9162

Epoch 00140: val_acc did not improve from 0.91620
Epoch 141/200
Learning rate:  1e-05
1563/1563 [==============================] - 45s 29ms/step - loss: 0.2012 - acc: 0.9764 - val_loss: 0.4172 - val_acc: 0.9151

Epoch 00141: val_acc did not improve from 0.91620
Epoch 142/200
Learning rate:  1e-05
1563/1563 [==============================] - 44s 28ms/step - loss

1563/1563 [==============================] - 46s 29ms/step - loss: 0.1932 - acc: 0.9777 - val_loss: 0.4214 - val_acc: 0.9159

Epoch 00175: val_acc did not improve from 0.91670
Epoch 176/200
Learning rate:  1e-06
1563/1563 [==============================] - 46s 29ms/step - loss: 0.1947 - acc: 0.9770 - val_loss: 0.4196 - val_acc: 0.9157

Epoch 00176: val_acc did not improve from 0.91670
Epoch 177/200
Learning rate:  1e-06
1563/1563 [==============================] - 46s 29ms/step - loss: 0.1931 - acc: 0.9776 - val_loss: 0.4208 - val_acc: 0.9157

Epoch 00177: val_acc did not improve from 0.91670
Epoch 178/200
Learning rate:  1e-06
1563/1563 [==============================] - 46s 29ms/step - loss: 0.1932 - acc: 0.9777 - val_loss: 0.4201 - val_acc: 0.9162

Epoch 00178: val_acc did not improve from 0.91670
Epoch 179/200
Learning rate:  1e-06
1563/1563 [==============================] - 46s 29ms/step - loss: 0.1942 - acc: 0.9769 - val_loss: 0.4188 - val_acc: 0.9165

Epoch 00179: val_acc did n

In [10]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 206us/step
Test loss: 0.4197410828590393
Test accuracy: 0.9163
